# `scinum` example

In [1]:
from scinum import Number, NOMINAL, UP, DOWN, ABS, REL

The examples below demonstrate

- Numbers and formatting
- Defining uncertainties
- Multiple uncertainties
- Automatic uncertainty propagation

### Numbers and formatting

In [2]:
n = Number(1.234, 0.2)
n

<Number at 0x11426e310, '1.234 +- 0.2'>

The uncertainty definition is absolute. See the examples with multiple uncertainties for relative uncertainty definitions.

The representation of numbers (`repr`) in jupyter notebooks uses latex-style formatting. Internally, [`Number.str()`](https://scinum.readthedocs.io/en/latest/#scinum.Number.str) is called, which - among others - accepts a `format` argument, defaulting to `"%s"` (configurable globally or per instance via `Number.default_format`). Let's change the format:

In [3]:
n.default_format = "%.3f"
n

<Number at 0x11426e310, '1.234 +- 0.200'>

In [4]:
# or
n.str("%.3f")

'1.234 +- 0.200'

### Defining uncertainties

Above, `n` is defined with a single, symmetric uncertainty. Here are some basic examples to access and play it:

In [5]:
# nominal value
print(n.nominal)
print(type(n.nominal))

1.234
<type 'float'>


In [6]:
# get the uncertainty
print(n.get_uncertainty())
print(n.get_uncertainty(direction=UP))
print(n.get_uncertainty(direction=DOWN))

(0.2, 0.2)
0.2
0.2


In [7]:
# get the nominal value, shifted by the uncertainty
print(n.get())      # nominal value
print(n.get(UP))    # up variation
print(n.get(DOWN))  # down variation

1.234
1.434
1.034


In [8]:
# some more advanved use-cases:

# 1. get the multiplicative factor that would scale the nomninal value to the UP/DOWN varied ones
print("absolute factors:")
print(n.get(UP, factor=True))
print(n.get(DOWN, factor=True))

# 2. get the factor to obtain the uncertainty only (i.e., the relative unceratinty)
# (this is, of course, more useful in case of multiple uncertainties, see below)
print("\nrelative factors:")
print(n.get(UP, factor=True, diff=True))
print(n.get(DOWN, factor=True, diff=True))

absolute factors:
1.16207455429
0.837925445705

relative factors:
0.162074554295
0.162074554295


There are also a few shorthands for the above methods:

In [9]:
# __call__ is forwarded to get()
print(n())
print(n(UP))

# u() is forwarded to get_uncertainty()
print(n.u())
print(n.u(direction=UP))

1.234
1.434
(0.2, 0.2)
0.2


### Multiple uncertainties

Let's create a number that has two uncertainties: `"stat"` and `"syst"`. The `"stat"` uncertainty is asymmetric, and the `"syst"` uncertainty is relative.

In [10]:
n = Number(8848, {
    "stat": (30, 20),   # absolute +30-20 uncertainty
    "syst": (REL, 0.5),  # relative +-50% uncertainty
})
n

<Number at 0x11425ccd0, '8848.0 +- 4424.0 (syst) +30.0-20.0 (stat)'>

Similar to above, we can access the uncertainties and shifted values with `get()` (or `__call__`) and `get_uncertainty()` (or `u()`). But this time, we can distinguish between the combined (in quadrature) value or the particular uncertainty sources:

In [11]:
# nominal value as before
print(n.nominal)

# get all uncertainties (stored absolute internally)
print(n.uncertainties)

8848.0
{'syst': (4424.0, 4424.0), 'stat': (30.0, 20.0)}


In [12]:
# get particular uncertainties
print(n.u("syst"))
print(n.u("stat"))
print(n.u("stat", direction=UP))

(4424.0, 4424.0)
(30.0, 20.0)
30.0


In [13]:
# get the nominal value, shifted by particular uncertainties
print(n(UP, "stat"))
print(n(DOWN, "syst"))

# compute the shifted value for both uncertainties, added in quadrature without correlation (default but configurable)
print(n(UP))

8878.0
4424.0
13272.1017167


As before, we can also access certain aspects of the uncertainties:

In [14]:
print("factors for particular uncertainties:")
print(n.get(UP, "stat", factor=True))
print(n.get(DOWN, "syst", factor=True))

print("\nfactors for the combined uncertainty:")
print(n.get(UP, factor=True))
print(n.get(DOWN, factor=True))

factors for particular uncertainties:
1.00339059675
0.5

factors for the combined uncertainty:
1.50001149601
0.499994890628


We can also apply some nice formatting:

In [15]:
print(n.str())
print(n.str("%.2f"))
print(n.str("%.2f", unit="m"))
print(n.str("%.2f", unit="m", force_asymmetric=True))
print(n.str("%.2f", unit="m", scientific=True))
print(n.str("%.2f", unit="m", si=True))
print(n.str("%.2f", unit="m", style="root"))

8848.0 +- 4424.0 (syst) +30.0-20.0 (stat)
8848.00 +- 4424.00 (syst) +30.00-20.00 (stat)
8848.00 +- 4424.00 (syst) +30.00-20.00 (stat) m
8848.00 +4424.00-4424.00 (syst) +30.00-20.00 (stat) m
8.85 +- 4.42 (syst) +0.03-0.02 (stat) x 1E3 m
8.85 +- 4.42 (syst) +0.03-0.02 (stat) km
8848.00 #pm 4424.00 #left(syst#right) ^{+30.00}_{-20.00} #left(stat#right) m


### Automatic uncertainty propagation

Let's continue working with the number `n` from above.

Uncertainty propagation works in a pythonic way:

In [16]:
n + 200

<Number at 0x11426e0d0, '9048.0 +- 4424.0 (syst) +30.0-20.0 (stat)'>

In [17]:
n / 2

<Number at 0x11426e410, '4424.0 +- 2212.0 (syst) +15.0-10.0 (stat)'>

In [18]:
n**0.5

<Number at 0x11426ed10, '94.0638081304 +- 23.5159520326 (syst) +0.159466220836-0.106310813891 (stat)'>

In cases such as the last one, formatting makes a lot of sense ...

In [19]:
(n**0.5).str("%.2f")

'94.06 +- 23.52 (syst) +0.16-0.11 (stat)'

More complex operations such as `exp`, `log`, `sin`, etc, are provided on the `ops` object, which mimics Python's `math` module. The benefit of the `ops` object is that all its operations are aware of Gaussian error propagation rules.

In [23]:
from scinum import ops

# change the default format for convenience
Number.default_format = "%.3f"

# compute the log of n
ops.log(n)

<Number at 0x114288390, '9.088 +- 0.500 (syst) +0.003-0.002 (stat)'>

The propagation is actually performed simultaneously per uncertainty source.

In [30]:
m = Number(5000, {"syst": 1000})

n + m

<Number at 0x114288850, '13848.000 +- 4535.612 (syst) +30.000-20.000 (stat)'>

In [31]:
n / m

<Number at 0x114288610, '1.770 +- 0.953 (syst) +0.006-0.004 (stat)'>

By default, equally named uncertainty sources are assumed to be fully correlated (rho = 1). You can configure the correlation in operations by explicitely using explicit methods on the number object.

In [32]:
n.add(m, rho=0.5, inplace=False)

<Number at 0x114288310, '13848.000 +- 4999.578 (syst) +30.000-20.000 (stat)'>

When you set `inplace` to `True` (the default), `n` is updated inplace.

In [34]:
n.add(m, rho=0.5)
n

<Number at 0x11425ccd0, '18848.000 +- 5567.347 (syst) +30.000-20.000 (stat)'>